In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [ ]:
# Load the uploaded CSV file
file_path = "C:/Users/Nabilah/OneDrive - Institut Teknologi Sepuluh Nopember/SEMESTER 7/Coffee Shop Sales.csv"
df = pd.read_csv(file_path)
df.set_index('transaction_date', inplace=True)
df.index = pd.to_datetime(df.index)
df

In [ ]:
# Group by store location and transaction date, then sum transaction quantity
transaction_sum_by_store = df.groupby(['store_location', df.index])['transaction_qty'].sum()

# Display the result
transaction_sum_by_store

In [ ]:
# Iterate through each store location and create a line plot
for store in transaction_sum_by_store.index.get_level_values(0).unique():
    plt.figure(figsize=(12, 6))
    plt.plot(transaction_sum_by_store[store].index, transaction_sum_by_store[store].values)
    plt.xlabel('Transaction Date')
    plt.ylabel('Total Transaction Quantity')
    plt.title(f'Total Transaction Quantity Over Time for {store}')
    plt.grid(True)
    plt.show()

In [ ]:
for store in transaction_sum_by_store.index.get_level_values(0).unique():
    # Extract the time series for the current store
    store_data = transaction_sum_by_store[store]

    # Perform ADF test
    result = adfuller(store_data)
    print(f'\nADF Test for {store}:')
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    # Interpret the results and apply differencing if necessary
    if result[1] > 0.05:
        print(f"The time series for {store} is likely non-stationary.")
        store_data_diff = store_data.diff().dropna()  # First-order differencing
        result_diff = adfuller(store_data_diff)
        print('\nADF Statistic after differencing: %f' % result_diff[0])
        print('p-value after differencing: %f' % result_diff[1])
        if result_diff[1] <= 0.05:
            print(f"Reject H0 - Time Series for {store} is stationary after differencing")
        else:
            print(f"Fail to reject H0 - Time Series for {store} is still non-stationary after differencing")
    else:
        print(f"The time series for {store} is likely stationary.")

In [ ]:
for store in transaction_sum_by_store.index.get_level_values(0).unique():
    store_data = transaction_sum_by_store[store]

    if result[1] > 0.05:
        store_data = store_data_diff

    fig, axes = plt.subplots(1, 2, figsize=(16, 3))
    plot_acf(store_data, lags=20, ax=axes[0])
    plot_pacf(store_data, lags=20, ax=axes[1])

    # Add title with spacing
    plt.suptitle(f'ACF and PACF for {store}', y=1.05)
    plt.show()

In [ ]:
#Cari Model Sarima Terbaik Untuk Masing-Masing Store
for store in transaction_sum_by_store.index.get_level_values(0).unique():
    # Extract data for the current store
    store_data = transaction_sum_by_store[store]

    # Split data into training and testing sets
    train_data, test_data = train_test_split(store_data, test_size=30, shuffle=False)

    # Define parameter ranges
    p = range(0, 4)
    d = [1]
    q = range(0, 2)
    P = range(0, 4)
    D = [1]
    Q = range(0, 2)
    s = [30]

    best_aic = float('inf')
    best_pdq = None
    best_seasonal_pdq = None
    best_model = None

    for param in itertools.product(p, d, q):
        for seasonal_param in itertools.product(P, D, Q, s):
          try:
              model = SARIMAX(train_data, order=param, seasonal_order=seasonal_param)
              results = model.fit()

              # Check p-values
              if all(pval < 0.05 for pval in results.pvalues):
                if results.aic < best_aic:
                    best_aic = results.aic
                    best_pdq = param
                    best_seasonal_pdq = seasonal_param
                    best_model = results
                    print(f"Store: {store}, Order: {param}, Seasonal Order: {seasonal_param}, AIC: {results.aic}, p-values: {results.pvalues}")
          except Exception as e:
              print(f"Error for {store}, order {param}, seasonal order {seasonal_param}: {e}")
              continue

    if best_model:
        print(f"\nBest SARIMAX order for {store} with all significant p-values: {best_pdq}x{best_seasonal_pdq}, AIC: {best_aic}")
        print("\nModel Summary:")
        print(best_model.summary())
    else:
        print(f"No SARIMAX order found for {store} with all significant p-values.")

# **ASTORIA**

In [ ]:
for store in transaction_sum_by_store.index.get_level_values(0).unique():
    if store == "Astoria":
        # Extract data for Astoria
        store_data = transaction_sum_by_store[store]

        # Split data into training and testing sets
        train_data, test_data = train_test_split(store_data, test_size=30, shuffle=False)

        # Model Building and Training
        try:
            p, d, q = 0, 1, 1  # Non-seasonal
            P, D, Q, s = 0, 1, 1, 30  # Seasonal
            model = SARIMAX(train_data, order=(p, d, q), seasonal_order=(P, D, Q, s))
            results = model.fit()
            print(f"\nSARIMA Model Summary for {store}:")
            print(results.summary())
            results.plot_diagnostics(figsize=(15, 12))
            plt.show()

            # Forecasting
            forecast_steps = len(test_data)
            forecast = results.get_forecast(steps=forecast_steps)
            forecast_mean = forecast.predicted_mean
            forecast_conf_int = forecast.conf_int()

            # Plot the forecast against the actual values
            plt.figure(figsize=(12, 6))
            plt.plot(train_data.index, train_data.values, label='Training Data')
            plt.plot(test_data.index, test_data.values, label='Actual Values')
            plt.plot(forecast_mean.index, forecast_mean.values, label='Forecast')
            plt.fill_between(forecast_conf_int.index, forecast_conf_int.iloc[:,0], forecast_conf_int.iloc[:,1], color='k', alpha=.2)
            plt.xlabel('Transaction Date')
            plt.ylabel('Total Transaction Quantity')
            plt.title(f'Forecast vs Actual for {store}')
            plt.legend()
            plt.grid(True)
            plt.show()

        except Exception as e:
            print(f"Error fitting SARIMA model for {store}: {e}")

In [ ]:
# Filter data untuk store "Astoria"
store_name = "Astoria"
if store_name in transaction_sum_by_store.index.get_level_values(0).unique():
    # Extract the time series for the selected store
    store_data = transaction_sum_by_store.loc[store_name]
    print(f"--- Processing data for store: {store_name} ---")

    # Split data into training and testing sets
    train_data, test_data = train_test_split(store_data, test_size=0.2, shuffle=False)

    # Model Building and Training (using training data)
    p, d, q = 0, 1, 1  # non-seasonal
    P, D, Q, s = 0, 1, 1, 30  # seasonal
    try:
        model = SARIMAX(train_data, order=(p, d, q), seasonal_order=(P, D, Q, s))
        results = model.fit()

        # Forecast future values
        future_steps = 30  # Number of steps to forecast into the future
        future_forecast = results.get_forecast(steps=future_steps)
        future_mean = future_forecast.predicted_mean
        future_conf_int = future_forecast.conf_int()

        # Create future dates for plotting
        last_date = store_data.index[-1]
        future_dates = pd.date_range(start=last_date + pd.DateOffset(1), periods=future_steps)

        # Plot future predictions
        plt.figure(figsize=(12, 6))
        plt.plot(store_data.index, store_data, label='Historical Data')
        plt.plot(future_dates, future_mean, label='Future Forecast')
        plt.fill_between(future_dates, future_conf_int.iloc[:, 0], future_conf_int.iloc[:, 1], color='gray', alpha=0.3, label='Confidence Interval')

        plt.xlabel('Transaction Date')
        plt.ylabel('Total Transaction Quantity')
        plt.title(f'Future Predictions for {store_name}')
        plt.legend()
        plt.grid(True)
        plt.show()

    except Exception as e:
        print(f"Error processing store {store_name}: {e}")
else:
    print(f"Store {store_name} not found in the data.")


In [ ]:
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))*100

for store in transaction_sum_by_store.index.get_level_values(0).unique():
    if store == "Astoria":
        # Extract data for the store
        store_data = transaction_sum_by_store[store]
        # Split data into training and testing sets
        train_data, test_data = train_test_split(store_data, test_size=30, shuffle=False)
        # Model Building and Training
        try:
            p, d, q = 0, 1, 1  # Non-seasonal
            P, D, Q, s = 0, 1, 1, 30  # Seasonal
            model = SARIMAX(train_data, order=(p, d, q), seasonal_order=(P, D, Q, s))
            results = model.fit()

            # Forecasting
            forecast_steps = len(test_data)
            forecast = results.get_forecast(steps=forecast_steps)
            forecast_mean = forecast.predicted_mean

            # Calculate MAPE
            store_mape = mape(test_data, forecast_mean)
            print(f"Store: {store}, MAPE: {store_mape}")

        except Exception as e:
            print(f"Error calculating MAPE for {store}: {e}")

# **HELL'S KITCHEN**

In [ ]:
for store in transaction_sum_by_store.index.get_level_values(0).unique():
    if store == "Hell's Kitchen":
        # Extract data for Hell's Kitchen
        store_data = transaction_sum_by_store[store]
        # Split data into training and testing sets
        train_data, test_data = train_test_split(store_data, test_size=30, shuffle=False)
        # Model Building and Training
        try:
            p, d, q = 0, 1, 1  # Non-seasonal
            P, D, Q, s = 0, 1, 1, 30  # Seasonal
            model = SARIMAX(train_data, order=(p, d, q), seasonal_order=(P, D, Q, s))
            results = model.fit()
            print(f"\nSARIMA Model Summary for {store}:")
            print(results.summary())
            results.plot_diagnostics(figsize=(15, 12))
            plt.show()

            # Forecasting
            forecast_steps = len(test_data)
            forecast = results.get_forecast(steps=forecast_steps)
            forecast_mean = forecast.predicted_mean
            forecast_conf_int = forecast.conf_int()

            # Plot the forecast against the actual values
            plt.figure(figsize=(12, 6))
            plt.plot(train_data.index, train_data.values, label='Training Data')
            plt.plot(test_data.index, test_data.values, label='Actual Values')
            plt.plot(forecast_mean.index, forecast_mean.values, label='Forecast')
            plt.fill_between(forecast_conf_int.index, forecast_conf_int.iloc[:,0], forecast_conf_int.iloc[:,1], color='k', alpha=.2)
            plt.xlabel('Transaction Date')
            plt.ylabel('Total Transaction Quantity')
            plt.title(f'Forecast vs Actual for {store}')
            plt.legend()
            plt.grid(True)
            plt.show()

        except Exception as e:
            print(f"Error fitting SARIMA model for {store}: {e}")

In [ ]:
# Filter data untuk store "Hell's Kitchen"
store_name = "Hell's Kitchen"
if store_name in transaction_sum_by_store.index.get_level_values(0).unique():
    # Extract the time series for the selected store
    store_data = transaction_sum_by_store.loc[store_name]
    print(f"--- Processing data for store: {store_name} ---")

    # Split data into training and testing sets
    train_data, test_data = train_test_split(store_data, test_size=0.2, shuffle=False)

    # Model Building and Training (using training data)
    p, d, q = 0, 1, 1  # non-seasonal
    P, D, Q, s = 0, 1, 1, 30  # seasonal
    try:
        model = SARIMAX(train_data, order=(p, d, q), seasonal_order=(P, D, Q, s))
        results = model.fit()

        # Forecast future values
        future_steps = 30  # Number of steps to forecast into the future
        future_forecast = results.get_forecast(steps=future_steps)
        future_mean = future_forecast.predicted_mean
        future_conf_int = future_forecast.conf_int()

        # Create future dates for plotting
        last_date = store_data.index[-1]
        future_dates = pd.date_range(start=last_date + pd.DateOffset(1), periods=future_steps)

        # Plot future predictions
        plt.figure(figsize=(12, 6))
        plt.plot(store_data.index, store_data, label='Historical Data')
        plt.plot(future_dates, future_mean, label='Future Forecast')
        plt.fill_between(future_dates, future_conf_int.iloc[:, 0], future_conf_int.iloc[:, 1], color='gray', alpha=0.3, label='Confidence Interval')

        plt.xlabel('Transaction Date')
        plt.ylabel('Total Transaction Quantity')
        plt.title(f'Future Predictions for {store_name}')
        plt.legend()
        plt.grid(True)
        plt.show()

    except Exception as e:
        print(f"Error processing store {store_name}: {e}")
else:
    print(f"Store {store_name} not found in the data.")


In [ ]:
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

for store in transaction_sum_by_store.index.get_level_values(0).unique():
    if store == "Hell's Kitchen":
        # Extract data for the store
        store_data = transaction_sum_by_store[store]
        # Split data into training and testing sets
        train_data, test_data = train_test_split(store_data, test_size=30, shuffle=False)
        # Model Building and Training
        try:
            p, d, q = 0, 1, 1  # Non-seasonal
            P, D, Q, s = 0, 1, 1, 30  # Seasonal
            model = SARIMAX(train_data, order=(p, d, q), seasonal_order=(P, D, Q, s))
            results = model.fit()

            # Forecasting
            forecast_steps = len(test_data)
            forecast = results.get_forecast(steps=forecast_steps)
            forecast_mean = forecast.predicted_mean

            # Calculate MAPE
            store_mape = mape(test_data, forecast_mean)
            print(f"Store: {store}, MAPE: {store_mape}")

        except Exception as e:
            print(f"Error calculating MAPE for {store}: {e}")

# **LOWER MANHATTAN**

In [ ]:
for store in transaction_sum_by_store.index.get_level_values(0).unique():
    if store == "Lower Manhattan":  # Only process data for Lower Manhattan
        # Extract data for Lower Manhattan
        store_data = transaction_sum_by_store[store]
        # Split data into training and testing sets
        train_data, test_data = train_test_split(store_data, test_size=30, shuffle=False)
        # Model Building and Training
        try:
            p, d, q = 1, 1, 1  # Non-seasonal
            P, D, Q, s = 0, 1, 1, 30  # Seasonal
            model = SARIMAX(train_data, order=(p, d, q), seasonal_order=(P, D, Q, s))
            results = model.fit()
            print(f"\nSARIMA Model Summary for {store}:")
            print(results.summary())
            results.plot_diagnostics(figsize=(15, 12))
            plt.show()

            # Forecasting
            forecast_steps = len(test_data)
            forecast = results.get_forecast(steps=forecast_steps)
            forecast_mean = forecast.predicted_mean
            forecast_conf_int = forecast.conf_int()

            # Plot the forecast against the actual values
            plt.figure(figsize=(12, 6))
            plt.plot(train_data.index, train_data.values, label='Training Data')
            plt.plot(test_data.index, test_data.values, label='Actual Values')
            plt.plot(forecast_mean.index, forecast_mean.values, label='Forecast')
            plt.fill_between(forecast_conf_int.index, forecast_conf_int.iloc[:,0], forecast_conf_int.iloc[:,1], color='k', alpha=.2)
            plt.xlabel('Transaction Date')
            plt.ylabel('Total Transaction Quantity')
            plt.title(f'Forecast vs Actual for {store}')
            plt.legend()
            plt.grid(True)
            plt.show()

        except Exception as e:
            print(f"Error fitting SARIMA model for {store}: {e}")

In [ ]:
# Filter data untuk store "Lower Manhattan"
store_name = "Lower Manhattan"  # Nama store yang ingin diproses
if store_name in transaction_sum_by_store.index.get_level_values(0).unique():
    # Extract the time series for the selected store
    store_data = transaction_sum_by_store.loc[store_name]
    print(f"--- Processing data for store: {store_name} ---")

    # Split data into training and testing sets
    train_data, test_data = train_test_split(store_data, test_size=30, shuffle=False)

    # Model Building and Training (using training data)
    p, d, q = 1, 1, 1  # non-seasonal
    P, D, Q, s = 0, 1, 1, 30  # seasonal
    try:
        model = SARIMAX(train_data, order=(p, d, q), seasonal_order=(P, D, Q, s))
        results = model.fit()

        # Forecast future values
        future_steps = 30  # Number of steps to forecast into the future
        future_forecast = results.get_forecast(steps=future_steps)
        future_mean = future_forecast.predicted_mean
        future_conf_int = future_forecast.conf_int()

        # Create future dates for plotting
        last_date = store_data.index[-1]
        future_dates = pd.date_range(start=last_date + pd.DateOffset(1), periods=future_steps)

        # Plot future predictions
        plt.figure(figsize=(12, 6))
        plt.plot(store_data.index, store_data, label='Historical Data')
        plt.plot(future_dates, future_mean, label='Future Forecast')
        plt.fill_between(future_dates, future_conf_int.iloc[:, 0], future_conf_int.iloc[:, 1], color='gray', alpha=0.5, label='Confidence Interval')

        plt.xlabel('Transaction Date')
        plt.ylabel('Total Transaction Quantity')
        plt.title(f'Future Predictions for {store_name}')
        plt.legend()
        plt.grid(True)
        plt.show()

    except Exception as e:
        print(f"Error processing store {store_name}: {e}")
else:
    print(f"Store {store_name} not found in the data.")


In [ ]:
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

for store in transaction_sum_by_store.index.get_level_values(0).unique():
    if store == "Lower Manhattan":  # Only calculate for Lower Manhattan
        # Extract data for the store
        store_data = transaction_sum_by_store[store]
        # Split data into training and testing sets
        train_data, test_data = train_test_split(store_data, test_size=30, shuffle=False)
        # Model Building and Training
        try:
            p, d, q = 1, 1, 1  # Non-seasonal
            P, D, Q, s = 0, 1, 1, 30  # Seasonal (assuming weekly seasonality)
            model = SARIMAX(train_data, order=(p, d, q), seasonal_order=(P, D, Q, s))
            results = model.fit()

            # Forecasting
            forecast_steps = len(test_data)
            forecast = results.get_forecast(steps=forecast_steps)
            forecast_mean = forecast.predicted_mean

            # Calculate MAPE
            store_mape = mape(test_data, forecast_mean)
            print(f"Store: {store}, MAPE: {store_mape}")

        except Exception as e:
            print(f"Error calculating MAPE for {store}: {e}")